In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statistics
from statistics import stdev
from datetime import datetime
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score, roc_auc_score, plot_roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_excel('Data Posbindu 1.xlsx')

In [3]:
df.dtypes

peserta_id                                  int64
jss_id                                     object
tanggal_lahir                      datetime64[ns]
pendidikan                                 object
pekerjaan                                  object
                                        ...      
rujuk_faskes_telinga                      float64
rujuk_masalah_kesehatan_telinga           float64
rujuk_penanganan_telinga                  float64
created_at                                 object
updated_at                                 object
Length: 62, dtype: object

In [4]:
df.isna().sum()

peserta_id                            0
jss_id                                0
tanggal_lahir                         0
pendidikan                            0
pekerjaan                             0
                                   ... 
rujuk_faskes_telinga               1942
rujuk_masalah_kesehatan_telinga    1942
rujuk_penanganan_telinga           1942
created_at                            0
updated_at                            0
Length: 62, dtype: int64

# preprocessing

In [5]:
data = df[['peserta_id','tanggal','nama_opd',
           'ptm_keluarga','tinggi_badan','berat_badan',
           'gula_berlebih','garam_berlebih','lemak_berlebih',
           'kurang_buah_sayur','kurang_aktivitas_fisik','konsumsi_alkohol']]
data.dtypes

peserta_id                  int64
tanggal                    object
nama_opd                   object
ptm_keluarga               object
tinggi_badan              float64
berat_badan               float64
gula_berlebih              object
garam_berlebih             object
lemak_berlebih             object
kurang_buah_sayur          object
kurang_aktivitas_fisik     object
konsumsi_alkohol           object
dtype: object

In [6]:
data['tanggal'] = pd.to_datetime(data['tanggal'])
data['bulan'] = pd.DatetimeIndex(data['tanggal']).month

<ipython-input-6-b5ae62dbecac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tanggal'] = pd.to_datetime(data['tanggal'])
<ipython-input-6-b5ae62dbecac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bulan'] = pd.DatetimeIndex(data['tanggal']).month


In [7]:
data = data.dropna()
data

,peserta_id,tanggal,nama_opd,ptm_keluarga,tinggi_badan,berat_badan,gula_berlebih,garam_berlebih,lemak_berlebih,kurang_buah_sayur,kurang_aktivitas_fisik,konsumsi_alkohol,bulan
2,76,2022-03-15,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,Kolesterol Tinggi,160.0,55.0,tidak,ya,tidak,ya,tidak,tidak,3
3,77,2022-03-15,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,"Penyakit Hipertensi,Penyakit Stroke",162.0,69.0,tidak,tidak,tidak,tidak,tidak,tidak,3
4,79,2022-03-16,BADAN KEPEGAWAIAN DAN PENGEMBANGAN SUMBER DAYA...,Penyakit Stroke,158.0,53.0,tidak,tidak,tidak,tidak,tidak,tidak,3
9,106,2022-03-17,DINAS SOSIAL TENAGA KERJA DAN TRANSMIGRASI,Penyakit Diabetes,158.0,65.0,tidak,tidak,tidak,ya,ya,tidak,3
11,112,2022-03-17,DINAS PERTANAHAN DAN TATA RUANG (KUNDHA NITI M...,Penyakit Diabetes,168.0,75.0,tidak,tidak,ya,ya,ya,tidak,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916,150,2022-08-25,INSPEKTORAT,Kolesterol Tinggi,182.0,84.0,tidak,tidak,ya,ya,ya,tidak,8
1918,302,2022-08-25,DINAS PEKERJAAN UMUM PERUMAHAN DAN KAWASAN PER...,Penyakit Diabetes,152.0,54.0,ya,ya,tidak,tidak,ya,tidak,8
1929,886,2022-08-25,BADAN PENANGGULANGAN BENCANA DAERAH,"Penyakit Diabetes,Penyakit Hipertensi",171.0,80.0,tidak,tidak,ya,ya,ya,tidak,8
1935,1641,2022-08-25,INSPEKTORAT,Penyakit Hipertensi,160.0,60.0,tidak,tidak,tidak,ya,tidak,tidak,8


In [8]:
std_bb = (statistics.stdev(data.berat_badan))
mean_bb = data.berat_badan.mean()
bb_max = mean_bb + (2*std_bb)
bb_min = mean_bb - (2*std_bb)

In [9]:
bb_max

93.72883758034834

In [10]:
bb_min

37.36356488371737

In [11]:
data[(data.berat_badan < bb_min)].index

Int64Index([349, 1052], dtype='int64')

In [12]:
data[(data.berat_badan > bb_max)].index

Int64Index([28, 45, 157, 223, 350, 729, 820, 857, 964, 1389, 1477, 1516, 1723,
            1936],
           dtype='int64')

In [13]:
for index, row in data.iterrows():
    if row['berat_badan'] < bb_min:
        data.berat_badan[349, 1052] = bb_min
    elif row['berat_badan'] > bb_max: 
        data.berat_badan[28, 45, 157, 223, 350, 729, 820, 857, 964, 1389, 1477, 1516, 1723, 1936] = bb_max

C:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-13-ca0b4bfaf6f5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.berat_badan[28, 45, 157, 223, 350, 729, 820, 857, 964, 1389, 1477, 1516, 1723, 1936] = bb_max
<ipython-input-13-ca0b4bfaf6f5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.berat_

In [14]:
std_tb = (statistics.stdev(data.tinggi_badan))
mean_tb = data.tinggi_badan.mean()
tb_max = mean_tb + (2*std_tb)
tb_min = mean_tb - (2*std_tb)

In [15]:
tb_max

175.26630899872794

In [16]:
tb_min

142.49960475897228

In [17]:
data[(data.tinggi_badan < tb_min)].index

Int64Index([1210, 1624], dtype='int64')

In [18]:
data[(data.tinggi_badan > tb_max)].index

Int64Index([28, 182, 218, 328, 857, 1389, 1681, 1916], dtype='int64')

In [19]:
for index, row in data.iterrows():
    if row['tinggi_badan'] < tb_min:
        data.tinggi_badan[1210, 1624] = tb_min
    elif row['tinggi_badan'] > tb_max: 
        data.tinggi_badan[28, 182, 218, 328, 857, 1389, 1681, 1916] = tb_max

C:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-19-249f43935704>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.tinggi_badan[28, 182, 218, 328, 857, 1389, 1681, 1916] = tb_max
<ipython-input-19-249f43935704>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.tinggi_badan[1210, 1624] = tb_min


In [20]:
#menjadikan tinggi bedan cm ke m
data['tinggi_badan'] = data['tinggi_badan']/100
data

,peserta_id,tanggal,nama_opd,ptm_keluarga,tinggi_badan,berat_badan,gula_berlebih,garam_berlebih,lemak_berlebih,kurang_buah_sayur,kurang_aktivitas_fisik,konsumsi_alkohol,bulan
2,76,2022-03-15,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,Kolesterol Tinggi,1.600000,55.000000,tidak,ya,tidak,ya,tidak,tidak,3
3,77,2022-03-15,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,"Penyakit Hipertensi,Penyakit Stroke",1.620000,69.000000,tidak,tidak,tidak,tidak,tidak,tidak,3
4,79,2022-03-16,BADAN KEPEGAWAIAN DAN PENGEMBANGAN SUMBER DAYA...,Penyakit Stroke,1.580000,53.000000,tidak,tidak,tidak,tidak,tidak,tidak,3
9,106,2022-03-17,DINAS SOSIAL TENAGA KERJA DAN TRANSMIGRASI,Penyakit Diabetes,1.580000,65.000000,tidak,tidak,tidak,ya,ya,tidak,3
11,112,2022-03-17,DINAS PERTANAHAN DAN TATA RUANG (KUNDHA NITI M...,Penyakit Diabetes,1.680000,75.000000,tidak,tidak,ya,ya,ya,tidak,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916,150,2022-08-25,INSPEKTORAT,Kolesterol Tinggi,1.752663,84.000000,tidak,tidak,ya,ya,ya,tidak,8
1918,302,2022-08-25,DINAS PEKERJAAN UMUM PERUMAHAN DAN KAWASAN PER...,Penyakit Diabetes,1.520000,54.000000,ya,ya,tidak,tidak,ya,tidak,8
1929,886,2022-08-25,BADAN PENANGGULANGAN BENCANA DAERAH,"Penyakit Diabetes,Penyakit Hipertensi",1.710000,80.000000,tidak,tidak,ya,ya,ya,tidak,8
1935,1641,2022-08-25,INSPEKTORAT,Penyakit Hipertensi,1.600000,60.000000,tidak,tidak,tidak,ya,tidak,tidak,8


In [21]:
#hitung value bmi
data['value_bmi'] = data['berat_badan']/(data['tinggi_badan']*data['tinggi_badan'])
data

,peserta_id,tanggal,nama_opd,ptm_keluarga,tinggi_badan,berat_badan,gula_berlebih,garam_berlebih,lemak_berlebih,kurang_buah_sayur,kurang_aktivitas_fisik,konsumsi_alkohol,bulan,value_bmi
2,76,2022-03-15,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,Kolesterol Tinggi,1.600000,55.000000,tidak,ya,tidak,ya,tidak,tidak,3,21.484375
3,77,2022-03-15,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,"Penyakit Hipertensi,Penyakit Stroke",1.620000,69.000000,tidak,tidak,tidak,tidak,tidak,tidak,3,26.291724
4,79,2022-03-16,BADAN KEPEGAWAIAN DAN PENGEMBANGAN SUMBER DAYA...,Penyakit Stroke,1.580000,53.000000,tidak,tidak,tidak,tidak,tidak,tidak,3,21.230572
9,106,2022-03-17,DINAS SOSIAL TENAGA KERJA DAN TRANSMIGRASI,Penyakit Diabetes,1.580000,65.000000,tidak,tidak,tidak,ya,ya,tidak,3,26.037494
11,112,2022-03-17,DINAS PERTANAHAN DAN TATA RUANG (KUNDHA NITI M...,Penyakit Diabetes,1.680000,75.000000,tidak,tidak,ya,ya,ya,tidak,3,26.573129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916,150,2022-08-25,INSPEKTORAT,Kolesterol Tinggi,1.752663,84.000000,tidak,tidak,ya,ya,ya,tidak,8,27.345282
1918,302,2022-08-25,DINAS PEKERJAAN UMUM PERUMAHAN DAN KAWASAN PER...,Penyakit Diabetes,1.520000,54.000000,ya,ya,tidak,tidak,ya,tidak,8,23.372576
1929,886,2022-08-25,BADAN PENANGGULANGAN BENCANA DAERAH,"Penyakit Diabetes,Penyakit Hipertensi",1.710000,80.000000,tidak,tidak,ya,ya,ya,tidak,8,27.358845
1935,1641,2022-08-25,INSPEKTORAT,Penyakit Hipertensi,1.600000,60.000000,tidak,tidak,tidak,ya,tidak,tidak,8,23.437500


# menampilkan data transformasi (seluruh data)

In [22]:
#transformasi
for index, row in data.iterrows():
    if row['value_bmi'] > 24.9:
        data.loc[index, 'status_kondisi'] = 'obesitas'
    elif row['value_bmi'] > 22.9:
        data.loc[index, 'status_kondisi'] = 'overweight'
    elif row['value_bmi'] > 18.5:
        data.loc[index, 'status_kondisi'] = 'ideal'
    elif row['value_bmi'] < 18.5:
        data.loc[index, 'status_kondisi'] = 'underweight'
data

,peserta_id,tanggal,nama_opd,ptm_keluarga,tinggi_badan,berat_badan,gula_berlebih,garam_berlebih,lemak_berlebih,kurang_buah_sayur,kurang_aktivitas_fisik,konsumsi_alkohol,bulan,value_bmi,status_kondisi
2,76,2022-03-15,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,Kolesterol Tinggi,1.600000,55.000000,tidak,ya,tidak,ya,tidak,tidak,3,21.484375,ideal
3,77,2022-03-15,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,"Penyakit Hipertensi,Penyakit Stroke",1.620000,69.000000,tidak,tidak,tidak,tidak,tidak,tidak,3,26.291724,obesitas
4,79,2022-03-16,BADAN KEPEGAWAIAN DAN PENGEMBANGAN SUMBER DAYA...,Penyakit Stroke,1.580000,53.000000,tidak,tidak,tidak,tidak,tidak,tidak,3,21.230572,ideal
9,106,2022-03-17,DINAS SOSIAL TENAGA KERJA DAN TRANSMIGRASI,Penyakit Diabetes,1.580000,65.000000,tidak,tidak,tidak,ya,ya,tidak,3,26.037494,obesitas
11,112,2022-03-17,DINAS PERTANAHAN DAN TATA RUANG (KUNDHA NITI M...,Penyakit Diabetes,1.680000,75.000000,tidak,tidak,ya,ya,ya,tidak,3,26.573129,obesitas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916,150,2022-08-25,INSPEKTORAT,Kolesterol Tinggi,1.752663,84.000000,tidak,tidak,ya,ya,ya,tidak,8,27.345282,obesitas
1918,302,2022-08-25,DINAS PEKERJAAN UMUM PERUMAHAN DAN KAWASAN PER...,Penyakit Diabetes,1.520000,54.000000,ya,ya,tidak,tidak,ya,tidak,8,23.372576,overweight
1929,886,2022-08-25,BADAN PENANGGULANGAN BENCANA DAERAH,"Penyakit Diabetes,Penyakit Hipertensi",1.710000,80.000000,tidak,tidak,ya,ya,ya,tidak,8,27.358845,obesitas
1935,1641,2022-08-25,INSPEKTORAT,Penyakit Hipertensi,1.600000,60.000000,tidak,tidak,tidak,ya,tidak,tidak,8,23.437500,overweight


In [23]:
#menghilangkan atribut data yang tidak terpakai
data = data[data.status_kondisi != 'overweight']
data = data[data.status_kondisi != 'underweight']
data

,peserta_id,tanggal,nama_opd,ptm_keluarga,tinggi_badan,berat_badan,gula_berlebih,garam_berlebih,lemak_berlebih,kurang_buah_sayur,kurang_aktivitas_fisik,konsumsi_alkohol,bulan,value_bmi,status_kondisi
2,76,2022-03-15,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,Kolesterol Tinggi,1.600000,55.000000,tidak,ya,tidak,ya,tidak,tidak,3,21.484375,ideal
3,77,2022-03-15,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,"Penyakit Hipertensi,Penyakit Stroke",1.620000,69.000000,tidak,tidak,tidak,tidak,tidak,tidak,3,26.291724,obesitas
4,79,2022-03-16,BADAN KEPEGAWAIAN DAN PENGEMBANGAN SUMBER DAYA...,Penyakit Stroke,1.580000,53.000000,tidak,tidak,tidak,tidak,tidak,tidak,3,21.230572,ideal
9,106,2022-03-17,DINAS SOSIAL TENAGA KERJA DAN TRANSMIGRASI,Penyakit Diabetes,1.580000,65.000000,tidak,tidak,tidak,ya,ya,tidak,3,26.037494,obesitas
11,112,2022-03-17,DINAS PERTANAHAN DAN TATA RUANG (KUNDHA NITI M...,Penyakit Diabetes,1.680000,75.000000,tidak,tidak,ya,ya,ya,tidak,3,26.573129,obesitas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1910,604,2022-08-25,INSPEKTORAT,"Penyakit Diabetes,Penyakit Hipertensi",1.560000,73.000000,tidak,tidak,ya,tidak,ya,tidak,8,29.996713,obesitas
1914,691,2022-08-25,INSPEKTORAT,"Penyakit Diabetes,Penyakit Hipertensi",1.630000,84.000000,tidak,ya,ya,tidak,ya,tidak,8,31.615793,obesitas
1916,150,2022-08-25,INSPEKTORAT,Kolesterol Tinggi,1.752663,84.000000,tidak,tidak,ya,ya,ya,tidak,8,27.345282,obesitas
1929,886,2022-08-25,BADAN PENANGGULANGAN BENCANA DAERAH,"Penyakit Diabetes,Penyakit Hipertensi",1.710000,80.000000,tidak,tidak,ya,ya,ya,tidak,8,27.358845,obesitas


In [24]:
#ubah kategori menjadi numerik untuk klasifikasi
number = LabelEncoder()
data['gula_berlebih'] = number.fit_transform(data['gula_berlebih'])
data['garam_berlebih'] = number.fit_transform(data['garam_berlebih'])
data['lemak_berlebih'] = number.fit_transform(data['lemak_berlebih'])
data['kurang_buah_sayur'] = number.fit_transform(data['kurang_buah_sayur'])
data['kurang_aktivitas_fisik'] = number.fit_transform(data['kurang_aktivitas_fisik'])
data['konsumsi_alkohol'] = number.fit_transform(data['konsumsi_alkohol'])
data['status_kondisi'] = number.fit_transform(data['status_kondisi'])

data

,peserta_id,tanggal,nama_opd,ptm_keluarga,tinggi_badan,berat_badan,gula_berlebih,garam_berlebih,lemak_berlebih,kurang_buah_sayur,kurang_aktivitas_fisik,konsumsi_alkohol,bulan,value_bmi,status_kondisi
2,76,2022-03-15,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,Kolesterol Tinggi,1.600000,55.000000,0,1,0,1,0,0,3,21.484375,0
3,77,2022-03-15,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,"Penyakit Hipertensi,Penyakit Stroke",1.620000,69.000000,0,0,0,0,0,0,3,26.291724,1
4,79,2022-03-16,BADAN KEPEGAWAIAN DAN PENGEMBANGAN SUMBER DAYA...,Penyakit Stroke,1.580000,53.000000,0,0,0,0,0,0,3,21.230572,0
9,106,2022-03-17,DINAS SOSIAL TENAGA KERJA DAN TRANSMIGRASI,Penyakit Diabetes,1.580000,65.000000,0,0,0,1,1,0,3,26.037494,1
11,112,2022-03-17,DINAS PERTANAHAN DAN TATA RUANG (KUNDHA NITI M...,Penyakit Diabetes,1.680000,75.000000,0,0,1,1,1,0,3,26.573129,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1910,604,2022-08-25,INSPEKTORAT,"Penyakit Diabetes,Penyakit Hipertensi",1.560000,73.000000,0,0,1,0,1,0,8,29.996713,1
1914,691,2022-08-25,INSPEKTORAT,"Penyakit Diabetes,Penyakit Hipertensi",1.630000,84.000000,0,1,1,0,1,0,8,31.615793,1
1916,150,2022-08-25,INSPEKTORAT,Kolesterol Tinggi,1.752663,84.000000,0,0,1,1,1,0,8,27.345282,1
1929,886,2022-08-25,BADAN PENANGGULANGAN BENCANA DAERAH,"Penyakit Diabetes,Penyakit Hipertensi",1.710000,80.000000,0,0,1,1,1,0,8,27.358845,1


# pembagian data berdasarkan bulan

In [25]:
data_bulan3 = data[(data.bulan == 3)]
data_bulan3 = data_bulan3[['peserta_id','bulan','nama_opd','gula_berlebih','garam_berlebih','lemak_berlebih',
                          'kurang_buah_sayur','kurang_aktivitas_fisik','konsumsi_alkohol','status_kondisi']]
data_bulan3

,peserta_id,bulan,nama_opd,gula_berlebih,garam_berlebih,lemak_berlebih,kurang_buah_sayur,kurang_aktivitas_fisik,konsumsi_alkohol,status_kondisi
2,76,3,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,0,1,0,1,0,0,0
3,77,3,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,0,0,0,0,0,0,1
4,79,3,BADAN KEPEGAWAIAN DAN PENGEMBANGAN SUMBER DAYA...,0,0,0,0,0,0,0
9,106,3,DINAS SOSIAL TENAGA KERJA DAN TRANSMIGRASI,0,0,0,1,1,0,1
11,112,3,DINAS PERTANAHAN DAN TATA RUANG (KUNDHA NITI M...,0,0,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...
469,658,3,SATUAN POLISI PAMONG PRAJA,0,0,0,1,1,0,1
478,666,3,SATUAN POLISI PAMONG PRAJA,1,0,1,0,0,0,1
494,308,3,DINAS KEPENDUDUKAN DAN PENCATATAN SIPIL,0,0,0,0,0,0,1
504,251,3,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,0,0,0,1,0,0,1


In [26]:
data_bulan5 = data[(data.bulan == 5)]
data_bulan5 = data_bulan5[['peserta_id','bulan','nama_opd','gula_berlebih','garam_berlebih','lemak_berlebih',
                          'kurang_buah_sayur','kurang_aktivitas_fisik','konsumsi_alkohol','status_kondisi']]
data_bulan5

,peserta_id,bulan,nama_opd,gula_berlebih,garam_berlebih,lemak_berlebih,kurang_buah_sayur,kurang_aktivitas_fisik,konsumsi_alkohol,status_kondisi
519,21,5,DINAS PENANAMAN MODAL DAN PELAYANAN TERPADU SA...,0,0,0,0,1,0,1
527,213,5,DINAS KESEHATAN,0,1,0,1,1,0,1
533,380,5,DINAS PEMBERDAYAAN PEREMPUAN PERLINDUNGAN ANAK...,0,0,0,1,1,0,1
540,926,5,DINAS KESEHATAN,0,0,0,0,1,0,1
545,919,5,DINAS KESEHATAN,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
1128,28,5,DINAS KESEHATAN,0,0,0,0,0,0,0
1133,653,5,DINAS PEKERJAAN UMUM PERUMAHAN DAN KAWASAN PER...,0,0,0,0,0,0,1
1137,594,5,DINAS KESEHATAN,0,0,0,1,1,0,1
1142,41,5,DINAS KESEHATAN,0,0,1,1,1,0,1


In [27]:
data_bulan6 = data[(data.bulan == 6)]
data_bulan6 = data_bulan6[['peserta_id','bulan','nama_opd','gula_berlebih','garam_berlebih','lemak_berlebih',
                          'kurang_buah_sayur','kurang_aktivitas_fisik','konsumsi_alkohol','status_kondisi']]
data_bulan6

,peserta_id,bulan,nama_opd,gula_berlebih,garam_berlebih,lemak_berlebih,kurang_buah_sayur,kurang_aktivitas_fisik,konsumsi_alkohol,status_kondisi
1149,1244,6,DINAS PERTANIAN DAN PANGAN,0,0,0,0,1,0,0
1150,700,6,DINAS PERTANIAN DAN PANGAN,0,0,0,0,0,0,1
1155,1241,6,DINAS PERTANIAN DAN PANGAN,0,0,0,1,1,0,1
1161,1264,6,DINAS PERTANIAN DAN PANGAN,0,0,0,0,1,0,0
1162,1262,6,DINAS PERTANIAN DAN PANGAN,0,0,0,0,0,0,0
1165,1267,6,DINAS PERTANIAN DAN PANGAN,1,1,0,1,1,0,1
1170,1273,6,DINAS PERTANIAN DAN PANGAN,0,0,0,0,0,0,0
1171,1277,6,DINAS PERTANIAN DAN PANGAN,0,0,0,0,1,0,0
1178,1290,6,DINAS PERTANIAN DAN PANGAN,0,0,0,0,0,0,1
1180,1291,6,DINAS PERTANIAN DAN PANGAN,0,0,0,1,1,0,1


In [28]:
data_bulan8 = data[(data.bulan == 8)]
data_bulan8 = data_bulan8[['peserta_id','bulan','nama_opd','gula_berlebih','garam_berlebih','lemak_berlebih',
                          'kurang_buah_sayur','kurang_aktivitas_fisik','konsumsi_alkohol','status_kondisi']]
data_bulan8

,peserta_id,bulan,nama_opd,gula_berlebih,garam_berlebih,lemak_berlebih,kurang_buah_sayur,kurang_aktivitas_fisik,konsumsi_alkohol,status_kondisi
1232,229,8,BAGIAN TATA PEMERINTAHAN,0,0,0,1,1,0,1
1233,264,8,BAGIAN TATA PEMERINTAHAN,0,0,0,1,0,0,0
1234,21,8,DINAS PENANAMAN MODAL DAN PELAYANAN TERPADU SA...,0,0,0,0,0,0,1
1242,305,8,DINAS PENANAMAN MODAL DAN PELAYANAN TERPADU SA...,1,0,1,1,0,0,1
1243,1113,8,DINAS KESEHATAN,0,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...
1910,604,8,INSPEKTORAT,0,0,1,0,1,0,1
1914,691,8,INSPEKTORAT,0,1,1,0,1,0,1
1916,150,8,INSPEKTORAT,0,0,1,1,1,0,1
1929,886,8,BADAN PENANGGULANGAN BENCANA DAERAH,0,0,1,1,1,0,1


# Klasifikasi

In [29]:
#klasifikasi bulan 3:
attributes3 = ['gula_berlebih','garam_berlebih','lemak_berlebih',
              'kurang_buah_sayur','kurang_aktivitas_fisik','konsumsi_alkohol']
target3 = ['status_kondisi']

x3 = data_bulan3[attributes3]
y3 = data_bulan3[target3]

model3_nb = GaussianNB()
model3_knn = KNeighborsClassifier(n_neighbors=1)
x3_train, x3_test, y3_train, y3_test = train_test_split(x3, y3, test_size=0.4)
model3_knn.fit(x3_train, y3_train)
model3_knn.score(x3_test, y3_test)

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.5853658536585366

In [30]:
#prediksi3 = model3.predict(x_test)
#confusion_matrix(prediksi3, y_test)

In [31]:
#print(classification_report(prediksi3, y_test))

In [32]:
#klasifikasi bulan 5:
attributes5 = ['gula_berlebih','garam_berlebih','lemak_berlebih',
              'kurang_buah_sayur','kurang_aktivitas_fisik','konsumsi_alkohol']
target5 = ['status_kondisi']

x5 = data_bulan5[attributes5]
y5 = data_bulan5[target5]

model5_nb = GaussianNB()
model5_knn = KNeighborsClassifier(n_neighbors=1)
x5_train, x5_test, y5_train, y5_test = train_test_split(x5, y5, test_size=0.4)
model5_knn.fit(x5_train, y5_train)
model5_knn.score(x5_test, y5_test)

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.6

In [33]:
#prediksi5 = model5.predict(x5_test)
#confusion_matrix(prediksi5, y5_test)

In [34]:
#print(classification_report(prediksi5, y5_test))

In [35]:
#klasifikasi bulan 6:
attributes6 = ['gula_berlebih','garam_berlebih','lemak_berlebih',
              'kurang_buah_sayur','kurang_aktivitas_fisik','konsumsi_alkohol']
target6 = ['status_kondisi']

x6 = data_bulan6[attributes6]
y6 = data_bulan6[target6]

model6_nb = GaussianNB()
model6_knn = KNeighborsClassifier(n_neighbors=1)
x6_train, x6_test, y6_train, y6_test = train_test_split(x6, y6, test_size=0.4)
model6_knn.fit(x6_train, y6_train)
model6_knn.score(x6_test, y6_test)

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.6666666666666666

In [36]:
#prediksi6 = model6.predict(x_test)
#confusion_matrix(prediksi6, y_test)

In [37]:
#print(classification_report(prediksi6, y_test))

In [38]:
#klasifikasi bulan 8:
attributes8 = ['gula_berlebih','garam_berlebih','lemak_berlebih',
              'kurang_buah_sayur','kurang_aktivitas_fisik','konsumsi_alkohol']
target8 = ['status_kondisi']

x8 = data_bulan8[attributes8]
y8 = data_bulan8[target8]

model8_nb = GaussianNB()
model8_knn = KNeighborsClassifier(n_neighbors=1)
x8_train, x8_test, y8_train, y8_test = train_test_split(x8, y8, test_size=0.4)
model8_knn.fit(x8_train, y8_train)
model8_knn.score(x8_test, y8_test)

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.6481481481481481

In [39]:
#prediksi8 = model8.predict(x_test)
#confusion_matrix(prediksi8, y_test)

In [40]:
#print(classification_report(prediksi8, y_test))

# Data untuk visual dengan status obesitas setiap bulan

In [41]:
data_bulan3[(data_bulan3.status_kondisi == 1)]

,peserta_id,bulan,nama_opd,gula_berlebih,garam_berlebih,lemak_berlebih,kurang_buah_sayur,kurang_aktivitas_fisik,konsumsi_alkohol,status_kondisi
3,77,3,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,0,0,0,0,0,0,1
9,106,3,DINAS SOSIAL TENAGA KERJA DAN TRANSMIGRASI,0,0,0,1,1,0,1
11,112,3,DINAS PERTANAHAN DAN TATA RUANG (KUNDHA NITI M...,0,0,1,1,1,0,1
12,116,3,BADAN PENGELOLAAN KEUANGAN DAN ASET DAERAH,0,0,0,1,1,0,1
13,39,3,BAGIAN ORGANISASI,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
469,658,3,SATUAN POLISI PAMONG PRAJA,0,0,0,1,1,0,1
478,666,3,SATUAN POLISI PAMONG PRAJA,1,0,1,0,0,0,1
494,308,3,DINAS KEPENDUDUKAN DAN PENCATATAN SIPIL,0,0,0,0,0,0,1
504,251,3,DINAS KOMUNIKASI INFORMATIKA DAN PERSANDIAN,0,0,0,1,0,0,1


In [42]:
data_bulan5[(data_bulan5.status_kondisi == 1)]

,peserta_id,bulan,nama_opd,gula_berlebih,garam_berlebih,lemak_berlebih,kurang_buah_sayur,kurang_aktivitas_fisik,konsumsi_alkohol,status_kondisi
519,21,5,DINAS PENANAMAN MODAL DAN PELAYANAN TERPADU SA...,0,0,0,0,1,0,1
527,213,5,DINAS KESEHATAN,0,1,0,1,1,0,1
533,380,5,DINAS PEMBERDAYAAN PEREMPUAN PERLINDUNGAN ANAK...,0,0,0,1,1,0,1
540,926,5,DINAS KESEHATAN,0,0,0,0,1,0,1
545,919,5,DINAS KESEHATAN,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
1127,1200,5,DINAS PERINDUSTRIAN KOPERASI DAN USAHA KECIL M...,1,1,0,0,1,0,1
1133,653,5,DINAS PEKERJAAN UMUM PERUMAHAN DAN KAWASAN PER...,0,0,0,0,0,0,1
1137,594,5,DINAS KESEHATAN,0,0,0,1,1,0,1
1142,41,5,DINAS KESEHATAN,0,0,1,1,1,0,1


In [43]:
data_bulan6[(data_bulan6.status_kondisi == 1)]

,peserta_id,bulan,nama_opd,gula_berlebih,garam_berlebih,lemak_berlebih,kurang_buah_sayur,kurang_aktivitas_fisik,konsumsi_alkohol,status_kondisi
1150,700,6,DINAS PERTANIAN DAN PANGAN,0,0,0,0,0,0,1
1155,1241,6,DINAS PERTANIAN DAN PANGAN,0,0,0,1,1,0,1
1165,1267,6,DINAS PERTANIAN DAN PANGAN,1,1,0,1,1,0,1
1178,1290,6,DINAS PERTANIAN DAN PANGAN,0,0,0,0,0,0,1
1180,1291,6,DINAS PERTANIAN DAN PANGAN,0,0,0,1,1,0,1
1181,1294,6,DINAS PERTANIAN DAN PANGAN,0,0,0,0,0,0,1
1206,1254,6,DINAS PERTANIAN DAN PANGAN,0,0,0,1,1,0,1
1210,1316,6,DINAS PERTANIAN DAN PANGAN,0,0,0,1,0,0,1
1222,1319,6,DINAS PERTANIAN DAN PANGAN,1,0,1,1,1,0,1
1227,1328,6,DINAS PERTANIAN DAN PANGAN,0,0,1,1,0,0,1


In [44]:
data_bulan8[(data_bulan8.status_kondisi == 1)]

,peserta_id,bulan,nama_opd,gula_berlebih,garam_berlebih,lemak_berlebih,kurang_buah_sayur,kurang_aktivitas_fisik,konsumsi_alkohol,status_kondisi
1232,229,8,BAGIAN TATA PEMERINTAHAN,0,0,0,1,1,0,1
1234,21,8,DINAS PENANAMAN MODAL DAN PELAYANAN TERPADU SA...,0,0,0,0,0,0,1
1242,305,8,DINAS PENANAMAN MODAL DAN PELAYANAN TERPADU SA...,1,0,1,1,0,0,1
1243,1113,8,DINAS KESEHATAN,0,0,0,1,1,0,1
1253,182,8,INSPEKTORAT,0,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...
1910,604,8,INSPEKTORAT,0,0,1,0,1,0,1
1914,691,8,INSPEKTORAT,0,1,1,0,1,0,1
1916,150,8,INSPEKTORAT,0,0,1,1,1,0,1
1929,886,8,BADAN PENANGGULANGAN BENCANA DAERAH,0,0,1,1,1,0,1
